<a href="https://colab.research.google.com/github/leticiaschaves/leticiaschaves/blob/data-projects/LESSON_03_basic_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício de Spark básico

#### Use o arquivo walmart_stock.csv para responder e completar as tarefas abaixo:

#### Inicie uma sessão Pyspark simples:

In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 59.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=15fa2d244b86a3683218727a6d3cbb9c33c8f8f0f4914e3a041033d8c8d2186c
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("teste").getOrCreate()

In [ ]:
import pyspark.sql.functions as F

In [ ]:
from pyspark.sql.types import StringType,BooleanType,DateType

#### Carregue o arquivo CSV *Walmart Stock*.

In [ ]:
dfWalmart = spark.read.csv("/content/walmart_stock.csv", header=True)

#### Quais são os nomes das colunas?

In [ ]:
dfWalmart.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### Como é o *schema* do dataframe?

In [ ]:
dfWalmart.printSchema

<bound method DataFrame.printSchema of DataFrame[Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]>

#### Imprima as 5 primeiras linhas:

In [ ]:
dfWalmart.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [ ]:
df = dfWalmart

In [ ]:
x = list(df.columns)

In [ ]:
x.remove('Date')

In [ ]:
x

['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [ ]:
for coluna in x: 
  df = df.withColumn(coluna, F.col(coluna).cast('float'))

In [ ]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: float (nullable = true)
 |-- Adj Close: float (nullable = true)



#### Use describe() para saber mais sobre o dataframe

In [ ]:
df.describe()

DataFrame[summary: string, Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

#### Qual é a data onde a coluna "High" tem o maior valor?

In [ ]:
df.orderBy('High').show(1)

+----------+-----+-----+----+-----+---------+---------+
|      Date| Open| High| Low|Close|   Volume|Adj Close|
+----------+-----+-----+----+-----+---------+---------+
|2015-11-13|56.74|57.06|56.3|56.42|1.25149E7|53.975582|
+----------+-----+-----+----+-----+---------+---------+
only showing top 1 row



#### Qual a média da coluna "Close"?

In [ ]:
df.select(F.avg('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844997363553|
+-----------------+



#### Qual o mínimo e o máximo da coluna "Volume"?

In [ ]:
df.select(F.min('Volume'), F.max('Volume')).show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|  2094900.0|8.0898096E7|
+-----------+-----------+



#### Em quantos dias a coluna "Close" foi menor que 60 dólares?

In [ ]:
df.filter(df['Close']<60).count()

81

#### Qual a porcentagem de dias onde a coluna "High" foi maior que 80 dólares?
Ou seja, $\frac{dias\_onde\_High > 80}{total\_de\_dias}$

In [ ]:
df.filter(df['High']>80).count()/df.count()

0.09141494435612083

#### Calcule a correlação de Pearson entre as colunas "High" e "Volume":
[Dica](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.corr.html)

In [ ]:
df.select(F.corr(F.col('High'), F.col('Volume'))).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326095027024|
+-------------------+



#### Qual é o máximo obtido na coluna "High" por ano?

In [ ]:
df.withColumn('Ano', F.year(F.col('Date'))).groupBy('Ano').max().show()

+----+---------+---------+--------+----------+-----------+--------------+--------+
| Ano|max(Open)|max(High)|max(Low)|max(Close)|max(Volume)|max(Adj Close)|max(Ano)|
+----+---------+---------+--------+----------+-----------+--------------+--------+
|2015|     90.8|    90.97|   89.25|     90.47|8.0898096E7|     84.914215|    2015|
|2013|    81.21|    81.37|   80.82|     81.21|  2.56837E7|      73.92987|    2013|
|2014|    87.08|    88.09|   86.48|     87.54|  2.28124E7|      81.70768|    2014|
|2012|     77.6|     77.6|   76.69|     77.15|  3.80073E7|     68.568375|    2012|
|2016|     74.5|    75.19|   73.63|      74.3|  3.50767E7|      73.23352|    2016|
+----+---------+---------+--------+----------+-----------+--------------+--------+



#### Qual a média da coluna "Close" para cada mês (para todos os anos)?


In [ ]:
df.withColumn('Mes', F.month(F.col('Date'))).groupBy('Mes').avg().select(['Mes', 'avg(Close)']).show()

+---+-----------------+
|Mes|       avg(Close)|
+---+-----------------+
| 12|72.84792482628012|
|  1| 71.4480196131338|
|  6| 72.4953774506191|
|  3|71.77794376266337|
|  5|72.30971685445533|
|  9|72.18411782208611|
|  4|72.97361900692894|
|  8| 73.0298185521906|
|  7|74.43971944078106|
| 10| 71.5785454489968|
| 11|72.11108927207418|
|  2|71.30680438169499|
+---+-----------------+



# Bom trabalho!